In [1]:
from utils.utils import *
from scripts.config import DefaultArgsNamespace
import torch
import torch.nn as nn
import torchvision.models as models
from datautils.ems import *
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
import wandb

import warnings
warnings.filterwarnings("ignore", message="Accurate seek is not implemented for pyav backend")




device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

args = DefaultArgsNamespace()

train_loader, val_loader, test_loader, train_class_stats, val_class_stats = eee_get_dataloaders(args)

# # get input feature  size
# dataiter = next(iter(train_loader))
# batch =  dataiter
args.dataloader_params["train_class_stats"] = train_class_stats
args.dataloader_params["val_class_stats"] = val_class_stats


# # Access the parsed arguments
model, optimizer, criterion, device = init_model(args)# verbose_mode = args.verbose
model = model.to(device)
scheduler = StepLR(optimizer, step_size=args.learning_params["lr_drop"], gamma=0.1)  # adjust parameters as needed

dummy_input = torch.randn(1, 120, 1024) # batch, num_frames, channels, height, width

dummy_input = dummy_input.to(args.device)

dummy_output = model(dummy_input)   

print(dummy_output.shape)
# repeat the output to match sequence length
dummy_output = dummy_output.repeat(1, 120, 1)

print(dummy_output.shape)




/home/cjh9fw/conda/envs/egoems/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


********** ========== **********
Loading dataloader for Segmentation task
Splitting data to windows
Class stats: {'chest_compressions': 1053, 'approach_patient': 12, 'review_medications': 18, 'inquire_substance_use': 9, 'check_symptom_duration': 12, 'document_lkw_time': 8, 'load_patient_to_stretcher': 75, 'secure_patient_on_stretcher': 32, 'notify_hospital_of_stroke_alert': 8, 'transport': 116, 'check_responsiveness': 28, 'check_vision_deficits': 6, 'assess_neglect_signs': 1, 'check_blood_pressure': 66, 'inquire_medication_anticoagulants': 4, 'handoff_patient_to_hospital': 27, 'check_breathing': 24, 'request_assistance': 5, 'request_aed': 6, 'attach_defib_pads': 103, 'no_action': 35, 'check_pulse': 24, 'turn_on_aed': 34, 'clear_for_analysis': 83, 'clear_for_shock': 40, 'administer_shock_aed': 34, 'open_airway': 2, 'ventilate_patient': 140, 'assess_patient': 53, 'explain_procedure': 8, 'place_right_arm_lead': 9, 'place_left_arm_lead': 11, 'place_right_leg_lead': 8, 'place_left_leg_lead'

/home/cjh9fw/conda/envs/egoems/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/cjh9fw/conda/envs/egoems/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/cjh9fw/conda/envs/egoems/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: 

Training class counts:  {'chest_compressions': 1053, 'approach_patient': 12, 'review_medications': 18, 'inquire_substance_use': 9, 'check_symptom_duration': 12, 'document_lkw_time': 8, 'load_patient_to_stretcher': 75, 'secure_patient_on_stretcher': 32, 'notify_hospital_of_stroke_alert': 8, 'transport': 116, 'check_responsiveness': 28, 'check_vision_deficits': 6, 'assess_neglect_signs': 1, 'check_blood_pressure': 66, 'inquire_medication_anticoagulants': 4, 'handoff_patient_to_hospital': 27, 'check_breathing': 24, 'request_assistance': 5, 'request_aed': 6, 'attach_defib_pads': 103, 'no_action': 35, 'check_pulse': 24, 'turn_on_aed': 34, 'clear_for_analysis': 83, 'clear_for_shock': 40, 'administer_shock_aed': 34, 'open_airway': 2, 'ventilate_patient': 140, 'assess_patient': 53, 'explain_procedure': 8, 'place_right_arm_lead': 9, 'place_left_arm_lead': 11, 'place_right_leg_lead': 8, 'place_left_leg_lead': 9, 'place_v2_lead': 6, 'place_v1_lead': 8, 'place_v3_lead': 3, 'place_v6_lead': 9, 'pla

In [2]:
import sys
print(sys.executable)




/home/cjh9fw/conda/envs/egoems/bin/python


In [3]:
keysteps = args.dataloader_params['keysteps']
out_classes = len(keysteps)
print(out_classes)

35


In [4]:
modality = args.dataloader_params['modality']
print("Modality: ", modality)

# Find feature dimension
input, feature_size, label = preprocess(next(iter(train_loader)), args.dataloader_params['modality'], model, device=device)
print("Feature size: ", feature_size)

# print(input[:,0,-3:])

args.transformer_params['input_dim'] = feature_size
args.transformer_params['output_dim'] = out_classes

model, optimizer, criterion, device = init_model(args)# verbose_mode = args.verbose
model = model.to(device)
scheduler = StepLR(optimizer, step_size=args.learning_params["lr_drop"], gamma=0.1)  # adjust parameters as needed

wandb_logger = wandb.init(
    # set the wandb project where this run will be logged
    project="EgoExoEMS",
    group="Keystep Recognition",
    mode="disabled",
    name="Testing on EgoExoEMS with I3D RGB Features - ICRA Model",
    notes="initial attempt ICRA model with I3D RGB features",
    config={
    "args": args,
    }
)



Modality:  ['audio']
Input shape: torch.Size([1, 155648, 2])
Converted mono shape: torch.Size([1, 155648])
Processed input_values shape: torch.Size([1, 155648])
Extracted Wav2Vec2 features: torch.Size([1, 486, 768])
Wav2Vec feature shape:  torch.Size([1, 486, 768])
Feature size:  768


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training class counts:  {'chest_compressions': 441, 'approach_patient': 0, 'check_responsiveness': 0, 'check_pulse': 0, 'check_breathing': 0, 'request_aed': 0, 'request_assistance': 0, 'turn_on_aed': 0, 'attach_defib_pads': 0, 'clear_for_analysis': 0, 'clear_for_shock': 0, 'administer_shock_aed': 0, 'open_airway': 0, 'place_bvm': 0, 'ventilate_patient': 0, 'no_action': 0, 'assess_patient': 0, 'explain_procedure': 0, 'shave_patient': 0, 'place_left_arm_lead': 0, 'place_right_arm_lead': 0, 'place_left_leg_lead': 0, 'place_right_leg_lead': 0, 'place_v1_lead': 0, 'place_v2_lead': 0, 'place_v3_lead': 0, 'place_v4_lead': 0, 'place_v5_lead': 0, 'place_v6_lead': 0, 'ask_patient_age_sex': 0, 'request_patient_to_not_move': 0, 'turn_on_ecg': 0, 'connect_leads_to_ecg': 0, 'obtain_ecg_recording': 0, 'interpret_and_report': 0}
Validation class counts:  {'chest_compressions': 283}
Class counts:  tensor([  1.,   1.,   1.,   1., 441.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,
          1.,   1.,   1., 

In [8]:
# # # print one batch

for i, batch in enumerate(train_loader):
    print(batch['smartwatch'].shape)
    input,feature_size, label = preprocess(batch, args.dataloader_params['modality'], model, device=device, task='segmentation')
    print("Model input:",input.shape)
    output = model(input)
    
    # print("Model output:",output.shape)
    # print("Label:",label.shape)
    # Get the majority label along the sequence length dimension
    
    # print("Majority label:", majority_label.shape)
    
    print("Model prediction:",output.argmax(dim=1))
    print("Ground truth:",label.item())   
    # Adjust output shape if needed to match the label shape before computing loss
    # Assuming you want to compute the loss between majority_label and pred, no need to repeat output now
    loss = criterion(output, label)  # Use majority label for loss computation
    
    print("Loss:",loss.item())
    break

# results_dir = './results'
# # # # Train the model
# for epoch in range(args.learning_params["epochs"]):
#     train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device, wandb_logger, modality=modality)
#     val_loss = validate(model, val_loader, criterion, device, wandb_logger, modality=modality)
#     scheduler.step()
#     print(f"Epoch: {epoch}, Train Loss: {train_loss}, Val Loss: {val_loss}")
#     results = test_model(model, test_loader, criterion, device, wandb_logger, modality=modality, epoch=epoch, results_dir=results_dir)
#     print(results)
#     break

# dummy_input = torch.randn(1, 3, 1024) # batch, num_frames, channels, height, width
# dummy_input = dummy_input.to(args.device)
# pred = model(dummy_input)
# print(pred.shape)


torch.Size([1, 0])
Input shape: torch.Size([1, 192512, 2])
Converted mono shape: torch.Size([1, 192512])
Processed input_values shape: torch.Size([1, 192512])
Extracted Wav2Vec2 features: torch.Size([1, 601, 768])
Wav2Vec feature shape:  torch.Size([1, 601, 768])
Model input: torch.Size([1, 601, 768])
Model prediction: tensor([30], device='cuda:0')
Ground truth: 4
Loss: 3.492825984954834


In [5]:
import torch
import torch.nn as nn

class CNN_Encoder(nn.Module):
    def __init__(self, feature_dim, out_channels, kernel_size=3):
        super(CNN_Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(in_channels=feature_dim, out_channels=512, kernel_size=kernel_size, stride=1, padding=kernel_size//2),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),  # No change in temporal dimension
            nn.Conv1d(in_channels=512, out_channels=256, kernel_size=kernel_size, stride=1, padding=kernel_size//2),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),  # No change in temporal dimension
            nn.Conv1d(in_channels=256, out_channels=out_channels, kernel_size=kernel_size, stride=1, padding=kernel_size//2),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)  # No change in temporal dimension
        )

    def forward(self, x):
        # x: [batch, seq_len, feature_dim] -> need to permute for Conv1d
        x = x.permute(0, 2, 1)  # [batch, feature_dim, seq_len]

        x = self.encoder(x)
        # x = torch.mean(x, dim=-1)  # Global average pooling
        return x

feature_size = 1024
hidden_dim = 256
cnn_encoder = CNN_Encoder(feature_size, hidden_dim)

# Test the model
dummy_input = torch.randn(1, 30, 1024)  # batch, seq_len, feature_dim

pred = cnn_encoder(dummy_input)

print(pred.shape)  # [batch, 1]




torch.Size([1, 256, 3])
